In [1]:
model_folder='/home/mara/multitask_adversarial/results/BASEL/'

In [2]:
import keras
keras.__version__
import tensorflow as tf
from sklearn.metrics import accuracy_score
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.visible_device_list = '0'
keras.backend.set_session(tf.Session(config=config))

Using TensorFlow backend.


In [3]:
'../../doc/data_shuffle.csv'

'../../doc/data_shuffle.csv'

In [78]:
## Loading OS libraries to configure server preferences
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import warnings
warnings.filterwarnings("ignore")
import setproctitle
SERVER_NAME = 'ultrafast'
EXPERIMENT_TYPE='test_baseline'
import time
import sys
import shutil
## Adding PROCESS_UC1 utilities
sys.path.append('../../lib/TASK_2_UC1/')
from models import *
from util import otsu_thresholding
from extract_xml import *
from functions import *                   
sys.path.append('../../lib/')
from mlta import *
import math
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.visible_device_list = '0'
keras.backend.set_session(tf.Session(config=config))

verbose=1 

cam16 = hd.File('/home/mara/adversarialMICCAI/data/ultrafast/cam16_500/patches.h5py',  'r', libver='latest', swmr=True)
all500 = hd.File('/home/mara/adversarialMICCAI/data/ultrafast/all500/patches.h5py',  'r', libver='latest', swmr=True)
extra17 = hd.File('/home/mara/adversarialMICCAI/data/ultrafast/extra17/patches.h5py',  'r', libver='latest', swmr=True)
tumor_extra17=hd.File('/home/mara/adversarialMICCAI/data/ultrafast/1129-1155/patches.h5py', 'r', libver='latest', swmr=True)
test2 = hd.File('/mnt/nas2/results/IntermediateResults/Camelyon/ultrafast/test_data2/patches.hdf5', 'r', libver='latest', swmr=True)
pannuke= hd.File('/mnt/nas2/results/IntermediateResults/Camelyon/pannuke/patches_fix.hdf5', 'r', libver='latest', swmr=True)

global data
data={'cam16':cam16,'all500':all500,'extra17':extra17, 'tumor_extra17':tumor_extra17, 'test_data2': test2, 'pannuke':pannuke}
global concept_db
concept_db = hd.File('../../data/normalized_cmeasures/concept_values.h5py','r')
# Note: nuclei_concepts not supported yet
#global nuclei_concepts
#nuclei_concepts=hd.File('/mnt/nas2/results/IntermediateResults/Mara/MICCAI2020/normalized.hdf5','r')

#SYSTEM CONFIGS 
CONFIG_FILE = 'doc/config.cfg'
COLOR = True
global new_folder
new_folder=folder_name=model_folder
BATCH_SIZE = 32

seed=1
print seed

# SET PROCESS TITLE
setproctitle.setproctitle('UC1_{}'.format(EXPERIMENT_TYPE))

# SET SEED
np.random.seed(seed)
tf.set_random_seed(seed)

# DATA SPLIT CSVs 
train_csv=open('/mnt/nas2/results/IntermediateResults/Camelyon/train_shuffle.csv', 'r') # How is the encoding of .csv files ?
val_csv=open('/mnt/nas2/results/IntermediateResults/Camelyon/val_shuffle.csv', 'r')
test_csv=open('/mnt/nas2/results/IntermediateResults/Camelyon/test_shuffle.csv', 'r')
train_list=train_csv.readlines()
val_list=val_csv.readlines()
test_list=test_csv.readlines()
test2_csv = open('/mnt/nas2/results/IntermediateResults/Camelyon/test2_shuffle.csv', 'r')
test2_list=test2_csv.readlines()
test2_csv.close()
train_csv.close()
val_csv.close()
test_csv.close()
#data_csv=open('/mnt/nas2/results/IntermediateResults/Camelyon/data_shuffle.csv', 'r')
#data_csv=open('./data/train.csv', 'r')
data_csv=open('/mnt/nas2/results/IntermediateResults/Camelyon/pannuke/pannuke_train_shuffled.csv', 'r')
data_list=data_csv.readlines()
data_csv.close()

# STAIN NORMALIZATION
def get_normalizer(patch, save_folder=''):
    normalizer = ReinhardNormalizer()
    normalizer.fit(patch)
    np.save('{}/normalizer'.format(save_folder),normalizer)
    np.save('{}/normalizing_patch'.format(save_folder), patch)
    print('Normalisers saved to disk.')
    return normalizer

def normalize_patch(patch, normalizer):
    return np.float64(normalizer.transform(np.uint8(patch)))

1


In [5]:
# LOAD DATA NORMALIZER
global normalizer
db_name, entry_path, patch_no = get_keys(data_list[0])
normalization_reference_patch = data[db_name][entry_path][patch_no]
normalizer = get_normalizer(normalization_reference_patch, save_folder=new_folder)

"""
Building baseline model 
"""
#
# MODEL: BASELINE
base_model = keras.applications.inception_v3.InceptionV3(include_top=False, weights='imagenet', input_shape=(224,224,3))
#base_model = keras.applications.resnet50.ResNet50(include_top=True, weights='imagenet')
#base_model = pretrained(ResNet50)((224,224,3),1000)
#base_model = wide_resnet(50, 2048)((224,224,3), (1))
feature_output=base_model.layers[-1].output
feature_output = keras.layers.GlobalAveragePooling2D()(feature_output)
feature_output = Dense(2048, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01), name='finetuned_features1')(feature_output)
feature_output = Dense(512, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01), name='finetuned_features2')(feature_output)
feature_output = Dense(256, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01), name='finetuned_features3')(feature_output)
finetuning = Dense(1,name='predictions')(feature_output)
#regression_output = keras.layers.Dense(1, activation = keras.layers.Activation('linear'), name='concept_regressor')(feature_output)
model = Model(input=base_model.input, output=[finetuning])

Using brightness standardization
Normalisers saved to disk.


In [73]:
"""
Batch generators: 
They load a patch list: a list of file names and paths. 
They use the list to create a batch of 32 samples. 
"""
class Generator(Sequence):
    # Class is a dataset wrapper for better training performance
    def __init__(self, patch_list, batch_size=32):
        #self.x, self.y = x_set, y_set
        self.patch_list=patch_list
        self.batch_size = batch_size
        self.num_samples=len(patch_list)
    def __len__(self):
        return self.num_samples/self.batch_size
    def __getitem__(self, idx):
        while True:
            offset = 0
            batch_size=self.batch_size
            for offset in range(0,self.num_samples, self.batch_size):
                batch_x = []
                batch_y = []
                #batch_ones=[]
                #batch_noise=[]
                batch_contrast = []
                #batch_domain = []
                #batch_n_area = [] # nuclei average area
                #batch_n_count = []
                batch_samples=self.patch_list[offset:offset+batch_size]
                for line in batch_samples:
                    db_name, entry_path, patch_no = get_keys(line)
                    patch=data[db_name][entry_path][patch_no]
                    patch=normalize_patch(patch, normalizer)
                    patch=keras.applications.inception_v3.preprocess_input(patch) #removed bc of BNorm
                    #patch=keras.applications.resnet50.preprocess_input(patch) 
                    label = get_class(line, entry_path) # is there a problem with get_class ?
                    batch_x.append(patch)
                    batch_y.append(label)
                batch_x=np.asarray(batch_x, dtype=np.float32)
                batch_y=np.asarray(batch_y, dtype=np.float32)
                batch_cm=np.ones(len(batch_y), dtype=np.float32)
            inds = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
            batch_x = [batch_x, batch_y, batch_cm]
            batch_y = batch_y
            return batch_x, batch_y
#def on_epoch_end(self):
    #    np.random.shuffle(self.indices)
def get_concept_measure(db_name, entry_path, patch_no, measure_type=''):
    ### note: The measures in the file should have been scaled beforehand
    # to have zero mean and unit std
    
    path=db_name+'/'+entry_path+'/'+str(patch_no)+'/'+measure_type
    #import pdb; pdb.set_trace()
    #print path
    #print concept_db[path]
    #print concept_db[path][0]
    try:
        cm=concept_db[path][0]
        return cm
    except:
        print("[ERR]: {}, {}, {}, {} with path {}".format(db_name, entry_path, patch_no, measure_type, path))
        return 1.
    
def get_segmented_concept_measure(db_name, entry_path, patch_no, measure_type=''):
    ### note: The measures in the file should have been scaled beforehand
    # to have zero mean and unit std
    try:
        cm = nuclei_concepts[db_name+'/'+entry_path+'/'+str(patch_no)+'/'+measure_type][0]
    except:
        error_log.write('[get_segmented_concept_measure] {}, {}, {}, {}'.format(db_name, entry_path, patch_no, measure_type))
        print "[ERROR] Issue retreiving concept measure for {}, {}, {}, {}".format(db_name, entry_path, patch_no, measure_type)
        return 1.

# BATCH GENERATORS
def get_batch_data(patch_list, batch_size=32):
    num_samples=len(patch_list)
    while True:
        offset = 0
        for offset in range(0,num_samples, batch_size):
            batch_x = []
            batch_y = []
            batch_contrast=[]
            batch_samples=patch_list[offset:offset+batch_size]
            for line in batch_samples[:(num_samples//batch_size)*batch_size]:
                db_name, entry_path, patch_no = get_keys(line)
                patch=data[db_name][entry_path][patch_no]
                patch=normalize_patch(patch, normalizer)
                patch=keras.applications.inception_v3.preprocess_input(patch) 
                label = get_class(line, entry_path) 
                batch_x.append(patch)
                batch_y.append(label)
                # ONES
                #batch_ones.append(1.)
                # NOISE
                #batch_noise.append(np.random.normal(0.))
                # CONCEPT = contrast
                batch_contrast.append(get_concept_measure(db_name, entry_path, patch_no, measure_type='norm_contrast'))
                # CONCEPT = domain
                #batch_domain.append(get_domain(db_name, entry_path))
                # CONCEPT = nuclei area
                #batch_n_area.append(get_segmented_concept_measure(db_name, entry_path, patch_no, measure_type='area'))
                #batch_contrast.append(get_segmented_concept_measure(db_name, entry_path, patch_no, measure_type='area'))
                # CONCEPT = nuclei counts
                #batch_n_count.append(get_segmented_concept_measure(db_name, entry_path, patch_no, measure_type='count'))
                #batch_contrast.append(get_segmented_concept_measure(db_name, entry_path, patch_no, measure_type='count'))
            #batch_domain=keras.utils.to_categorical(batch_domain, num_classes=6)
            batch_x=np.asarray(batch_x, dtype=np.float32)
            batch_y=np.asarray(batch_y, dtype=np.float32)
            batch_cm=np.asarray(batch_contrast, dtype=np.float32) #ones(len(batch_y), dtype=np.float32)
            #batch_cm=np.ones(len(batch_y), dtype=np.float32)
            yield batch_x, batch_y#, batch_cm
            
def get_test_batch(patch_list, batch_size=32):
    num_samples=len(patch_list)
    while True:     
        for offset in range(0,num_samples, batch_size):
            batch_x = []
            batch_y = []
            batch_contrast = []
            batch_samples=patch_list[offset:offset+batch_size]
            for line in batch_samples:
                db_name, entry_path, patch_no = get_keys(line)
                
                patch=data[db_name][entry_path][patch_no]
                patch=normalize_patch(patch, normalizer)
                patch=keras.applications.inception_v3.preprocess_input(patch)
                #patch=keras.applications.resnet50.preprocess_input(patch)
                label = get_test_label(entry_path)
                #print db_name, entry_path, patch_no, label
                batch_x.append(patch)
                batch_y.append(label)
                #batch_ones.append(1.)
                # NOISE
                #batch_noise.append(np.random.normal(0.))
                # CONCEPT = contrast
                batch_contrast.append(get_concept_measure(db_name, entry_path, patch_no, measure_type='norm_contrast'))
                # CONCEPT = domain
                #batch_domain.append(get_domain(db_name, entry_path))
                # CONCEPT = nuclei area
                #batch_n_area.append(get_segmented_concept_measure(db_name, entry_path, patch_no, measure_type='area'))
                #batch_contrast.append(get_segmented_concept_measure(db_name, entry_path, patch_no, measure_type='area'))
                # CONCEPT = nuclei counts
                #batch_n_count.append(get_segmented_concept_measure(db_name, entry_path, patch_no, measure_type='count'))
                #batch_contrast.append(get_segmented_concept_measure(db_name, entry_path, patch_no, measure_type='count'))
            #batch_domain=keras.utils.to_categorical(batch_domain, num_classes=6)
            batch_x=np.asarray(batch_x, dtype=np.float32)
            batch_y=np.asarray(batch_y, dtype=np.float32)
            #batch_cm=np.ones(len(batch_y), dtype=np.float32)
            batch_cm=np.asarray(batch_contrast, dtype=np.float32) # np.ones(len(batch_y), dtype=np.float32)
            yield batch_x, batch_y#, batch_cm
            #yield np.asarray(batch_x, dtype=np.float32), np.asarray(batch_y, dtype=np.float32), np.ones(len(batch_y), dtype=np.float32)#, np.asarray(batch_cm, dtype=np.float32)


# In[2]:

In [7]:
def evaluate_model(d_list, model, batch_size=BATCH_SIZE):
    test_generator=get_test_batch(d_list, batch_size=batch_size)
    steps=len(d_list)//batch_size
    print steps
    initial_lr = 1e-4
    opt = keras.optimizers.SGD(lr=initial_lr, momentum=0.9, nesterov=True)
    model.compile(optimizer=opt, 
                 loss= [classifier_loss],
                   metrics= [my_accuracy])
    callbacks = []
    y_true=np.zeros(len(d_list))
    y_pred=np.zeros((len(d_list),1))
    N=0
    while N<len(d_list):
        x_b, y_b = test_generator.next()
        y_p_b = model.predict(x_b)
        y_true[N:N+len(y_b)]=y_b
        y_pred[N:N+len(y_p_b)]=y_p_b
        N+=len(y_p_b)
    sliced_y_pred = tf.sigmoid(y_pred)
    y_pred_rounded = K.round(sliced_y_pred)
    print 'accuracy: ', accuracy_score(y_pred_rounded.eval(session=tf.Session()), y_true)
    sliced_y_pred=sliced_y_pred.eval(session=tf.Session())
    auc_score=sklearn.metrics.roc_auc_score(y_true,sliced_y_pred)
    print 'auc: ', auc_score
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    y_true=np.reshape(y_true,(len(y_true),1))
    for i in range(1):
        fpr[i], tpr[i], _ = roc_curve(y_true[:, i], sliced_y_pred[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_true.ravel(), sliced_y_pred.ravel())
    roc_auc["micro"] = auc_score
    plot=False
    if plot:
        plt.figure()
        lw = 2
        plt.plot(fpr[0], tpr[0], color='darkorange',
                 lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[0])
        plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver operating characteristic example')
        plt.legend(loc="lower right")
        plt.show()
    return auc_score, y_pred_rounded

In [8]:
model.load_weights(model_folder+'/best_model.h5')

In [9]:
list_=test_list+test2_list
auc_score, y_pred_rounded=evaluate_model(list_,model)

116
accuracy:  0.7019386106623586
auc:  0.7870614607595342


In [10]:
def get_bootstrap_sample(data, n_samples=2):
    sample_=[data[i] for i in np.random.choice(len(data),n_samples)]
    #sample_=[data[i] for i in range(len(data))]
    return sample_

keras.backend.get_session().run(tf.global_variables_initializer())
model.load_weights(model_folder+'/best_model.h5')
aucs_i=[]
for i in range(50):
    test_list_b=get_bootstrap_sample(list_, n_samples=len(list_))
    roc_auc, _=evaluate_model(test_list_b,model)#, test_type='bootstrap_overall')
    aucs_i.append(roc_auc)
print "AUC avg (std): {} ({})".format(np.mean(aucs_i), np.std(aucs_i))

116
accuracy:  0.7035541195476576
auc:  0.7843508679851962
116
accuracy:  0.6976305869682283
auc:  0.7909975857989268
116
accuracy:  0.7272482498653743
auc:  0.8034849567935647
116
accuracy:  0.7011308562197092
auc:  0.7867221637687483
116
accuracy:  0.7113624124932687
auc:  0.7956770550891037
116
accuracy:  0.7094776521270867
auc:  0.7923959890995591
116
accuracy:  0.7035541195476576
auc:  0.7892464102295357
116
accuracy:  0.7014001077005924
auc:  0.7832763785052101
116
accuracy:  0.7089391491653204
auc:  0.7871577059040785
116


KeyboardInterrupt: 

In [ ]:
aucs_i

In [ ]:
#aucs_i
guided_aucs=[0.8097256593007709,
 0.8257677583461371,
 0.8284987543793056,
 0.8123987690905612,
 0.8122129193557766,
 0.8171528762435989,
 0.8203942669480053,
 0.801909346122524,
 0.8110533838973162,
 0.8192357512953368,
 0.8163210561918313,
 0.8049622015028518,
 0.8164248807485958,
 0.8261588579545603,
 0.8224349505654979,
 0.8198083365432877,
 0.8233338473508035,
 0.8151898394372206,
 0.8116935267976655,
 0.8230061920156451,
 0.80999705999706,
 0.8225011543919973,
 0.8194816813770406,
 0.8124183815864469,
 0.8216119126421779,
 0.8131742798952575,
 0.8133916206624894,
 0.8185327317394306,
 0.8132800425609821,
 0.826003441176462,
 0.81668630997849,
 0.8185735954190274,
 0.8031802696797476,
 0.8128920345130592,
 0.8152530177337852,
 0.8212378998659381,
 0.8154698586741791,
 0.8117851422186724,
 0.8090521202113915,
 0.8107352601298298,
 0.812407687849384,
 0.8090276887279646,
 0.8221827009936765,
 0.8134452910776464,
 0.8137599048373841,
 0.8127480241741258,
 0.812849532383189,
 0.8179884057229148,
 0.8306913580318795,
 0.8171719576042143]

In [11]:
import scipy.stats


In [ ]:
scipy.stats.ttest_ind(aucs_i, guided_aucs, equal_var=False)

In [ ]:
scipy.stats.wilcoxon(aucs_i, guided_aucs)

In [ ]:
scipy.stats.mannwhitneyu(aucs_i, guided_aucs)

In [ ]:
auc_score, y_pred_rounded=evaluate_model(test_list,model)

In [ ]:
evaluate_model(test2_list,model)

In [88]:
def get_batch_data_(patch_list, batch_size=32):
    print batch_size
    num_samples=len(patch_list)
    while True:
        offset = 0
        for offset in range(0,num_samples, batch_size):
            batch_x = []
            batch_y = []
            batch_samples=patch_list[offset:offset+batch_size]
            for line in batch_samples[:(num_samples//batch_size)*batch_size]:
                print line
                db_name, entry_path, patch_no = get_keys(line)
                patch=data[db_name][entry_path][patch_no]
                patch=normalize_patch(patch, normalizer)
                patch=keras.applications.inception_v3.preprocess_input(patch) 
                label = get_class(line, entry_path) 
                batch_x.append(patch)
                batch_y.append(label)
            batch_x=np.asarray(batch_x, dtype=np.float32)
            batch_y=np.asarray(batch_y, dtype=np.float32)
            generator_output=[batch_x, batch_y]
            
            for c in CONCEPT:
                batch_concept_values=[]
                for line in batch_samples[:(num_samples//batch_size)*batch_size]:
                    #print 'concept: ', c, line
                    db_name, entry_path, patch_no = get_keys(line)
                    batch_concept_values.append(get_concept_measure(db_name, entry_path, patch_no, measure_type=c))
                batch_concept_values=np.asarray(batch_concept_values, dtype=np.float32)
                generator_output.append(batch_concept_values)
            yield generator_output, None

In [91]:
CONCEPT=['ncount', 'narea', 'full_contrast']#, 'domain']
error_log=open('./baseline_test_log.txt', 'a')
test_generator = get_batch_data_(val_list, batch_size=32)

In [94]:
all_batches=
while i <= len(val_list)+32 :
    batch = test_generator.next()

1820

In [97]:
!pip install abc

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support pip 21.0 will remove support for this functionality.
ERROR: Could not find a version that satisfies the requirement abc (from versions: none)
ERROR: No matching distribution found for abc


In [96]:
!pip install rcvtool
import rcvtool

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support pip 21.0 will remove support for this functionality.
     |████████████████████████████████| 421.8 MB 21 kB/s s eta 0:00:012| 208.2 MB 112.5 MB/s eta 0:00:02| 210.5 MB 112.5 MB/s eta 0:00:02��█▏            | 252.1 MB 20.4 MB/s eta 0:00:09
     |████████████████████████████████| 3.8 MB 36.8 MB/s eta 0:00:01
     |████████████████████████████████| 448 kB 84.6 MB/s eta 0:00:01
     |████████████████████████████████| 146 kB 81.1 MB/s eta 0:00:01
  Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)


  Using cached requests_oauthlib-1.3.0-py2.py3-none-any.whl (23 kB)
  Using cached pyasn1-0.4.8-py2.py3-none-any.whl (77 kB)
  Using cached oauthlib-3.1.0-py2.py3-none-any.whl (147 kB)
  Created wheel for rcvtool: filename=rcvtool-0.1.5-py2-none-any.whl size=8715 sha256=2e46a2761f2e7e1c2012ed2575d5a74de442284c9505c6b38c98cacc9ec3b22f
  Stored in directory: /home/mara/.cache/pip/wheels/a0/1f/50/0a34a02215cf95403c129de91c16ca9e87d145dd798a8a9904
Successfully built rcvtool
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 1.8.0
    Uninstalling tensorboard-1.8.0:
      Successfully uninstalled tensorboard-1.8.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
ERROR: pip's legacy dependency resolver does not consider dependency conflicts when selecting packages. This behaviour is the source of the fo

ImportError: No module named abc

In [ ]:
def compute_rsquared(labels, predictions):
    errors = labels - predictions
    sum_squared_errors = np.sum(np.asarray([pow(errors[i],2) for i in range(len(errors))]))
    # total sum of squares, TTS
    average_y = np.mean(labels)
    total_errors = labels - average_y
    total_sum_squares = np.sum(np.asarray([pow(total_errors[i],2) for i in range(len(total_errors))]))
    #rsquared is 1-RSS/TTS
    rss_over_tts =   sum_squared_errors/total_sum_squares
    rsquared = 1-rss_over_tts
    return rsquared
def compute_mse(labels, predictions):
    errors = labels - predictions
    sum_squared_errors = np.sum(np.asarray([pow(errors[i],2) for i in range(len(errors))]))
    mse = sum_squared_errors / len(labels)
    return mse

In [ ]:
r2_i = compute_rsquared(all_cm_i, all_p_cm_i)
mse_i = compute_mse(all_cm_i, all_p_cm_i)
print 'Internal: ', r2_i, mse_i

In [ ]:
r2_e = compute_rsquared(all_cm_e, all_p_cm_e)
mse_e = compute_mse(all_cm_e, all_p_cm_e)
print 'External: ', r2_e, mse_e